# TPGshipのシミュレーション動画作成プログラム（2/4）

## 動作時のグラフ図作成編

ライブラリ読み込み

In [5]:
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scienceplots
import polars as pl
import numpy as np
import math
from numpy import linalg as LA
import cartopy.feature as cfeature
from PIL import Image
import matplotlib.patches as patches
from datetime import datetime, timedelta, timezone
from dateutil import tz


データファイル読み取り

台風＆TPGship経路データ

In [6]:
year = 2018

TPGship_data = pl.read_csv("output_Folder/TPGshipdata/" + str(year) + "_sim_operation_data+wind.csv")

typhoon_data = pl.read_csv("typhoon_data_6hour_intervals_verpl/table" + str(year) + "_6_interval.csv",encoding="shift-jis")

拠点と運搬船

In [7]:
stBASE_data = pl.read_csv("output_Folder/storageBASEdata/" + str(year) + "storageBASE.csv")
#spSHIP1_data = pl.read_csv("output_Folder/supportSHIPdata/" + str(year) + "supportSHIP1.csv")
#spSHIP2_data = pl.read_csv("output_Folder/supportSHIPdata/" + str(year) + "supportSHIP2.csv") 

TPGshipの動作時のグラフ画像化

In [8]:
import os


UTC = timezone(timedelta(hours =+ 0),"UTC")

# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder3", exist_ok=True)

#データの整理
totalgene = TPGship_data["YEARLY POWER GENERATION BALANCE"]
tg = []
for i in range(len(totalgene)):
    tg.append(totalgene[i]/10**9)

onboardene = TPGship_data["ONBOARD ENERGY STORAGE[Wh]"]
obe = []
for i in range(len(onboardene)):
    obe.append(onboardene[i]/10**9)

basestorage = stBASE_data["STORAGE[Wh]"]
base_data = []
for i in range(len(basestorage)):
    base_data.append(basestorage[i]/10**9)

day = TPGship_data["unixtime"]
daylist = []
for i in range(len(day)):
    daylist.append((day[i]-day[0])/86400)

for j in range(len(TPGship_data)):
#for j in range(3):
#    j_ori = j
#    j = j + 1320
    #地図の作成
    fig = plt.figure(figsize=(10,16)) # プロット領域の作成（matplotlib）
    ncols = 3
    nrows = 1

    #グラフの表示
    #今回は'science'と'ieee'を利用してみる
    plt.style.use(['science','ieee','high-vis','grid'])
    plt.rcParams["font.size"] = 20

    day_x = daylist[0:j]
    obe_y = obe[0:j]
    tg_y = tg[0:j]
    base_data_y = base_data[0:j]

    ax1 =  fig.add_subplot(3,1,1)
    ax1.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax1.set_ylim(ymin=0,ymax=75) #y軸の範囲を指定
    ax1.plot(day_x,obe_y,label = 'ONBOARD', linewidth=3)
    ax1.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax1.set(ylabel = 'Onboard Power Storage[GWh]') #y軸のラベル
    #ax1.tick_params(labelsize=18)
    
    ax2 =  fig.add_subplot(3,1,3)
    ax2.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax2.set_ylim(ymin=0,ymax=350) #y軸の範囲を指定
    ax2.plot(day_x,tg_y,label = 'TOTAL', linewidth=3)
    ax2.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax2.set(ylabel = 'Power Generation[GWh]') #y軸のラベル
    #plt.tick_params(labelsize=18)

    ax3 =  fig.add_subplot(3,1,2)
    ax3.set_xlim(xmin=0,xmax=365) #x軸の範囲を指定
    ax3.set_ylim(ymin=0,ymax=100) #y軸の範囲を指定
    ax3.plot(day_x,base_data_y,label = 'BASE OPERATION', linewidth=3)
    ax3.set(xlabel = 'Operation Time[Day]') #x軸のラベル
    ax3.set(ylabel = 'Electric at Storage Base[GWh]') #y軸のラベル

    #j = j_ori
    
    plt.savefig("png_Folder3/draw" + str(j) + ".png")
    plt.close(fig)
    
    